In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; print("Python kernel path:", sys.executable)

Python kernel path: /opt/homebrew/Caskroom/miniconda/base/envs/symbxai/bin/python


In [3]:
from symbxai.lrp.symbolic_xai import BERTSymbXAI
from symbxai.model.transformer import  bert_base_uncased_model
from dgl.data import SSTDataset

from symbxai.visualization.query_search import plot_quali_table
from IPython.display import display, HTML


import torch

from tqdm import tqdm
from functools import reduce

/opt/homebrew/Caskroom/miniconda/base/envs/symbxai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import matplotlib.pyplot as plt


import time, pickle, numpy

# from IPython.core.display import display, HTML
# from html2image import Html2Image
from symbxai.dataset.utils import load_sst_treebank

from symbxai.query_search.utils import calc_weights

from symbxai.query_search.utils import comp_all_harsanyi_sst, setup_queries, calc_attr_supp, calc_corr
import multiprocessing
from symbxai.dataset.utils import test_contr_conj, process_treeid2tokenid
from datasets import load_dataset

# Get NLP model

In [5]:
import transformers
sst_model = bert_base_uncased_model(
        pretrained_model_name_or_path='textattack/bert-base-uncased-SST-2' )

sst_model.eval()
pretrained_embeddings = sst_model.bert.embeddings
tokenizer = transformers.BertTokenizer.from_pretrained("textattack/bert-base-uncased-SST-2")

# Get SST Data

In [6]:
tree_dataset = SSTDataset(mode='train')
# sst_dataset = load_sst_treebank([sample_id])
vocab_words = list(tree_dataset.vocab.keys())

# sample_range = [138, 259, ] # 1481, 4113

# if False:
result_str = ''
count = 0
sample_range = []
for sample_id, tree_sample in enumerate(tree_dataset):
    ccout = test_contr_conj(tree_sample, vocab_words, verbose=False)
    
    if ccout == False: 
        continue
        
    sentence = ccout[5]
    lsent = ccout[4]
    sample = tokenizer(sentence, return_tensors="pt")
    tokens = tokenizer.convert_ids_to_tokens(sample['input_ids'].squeeze())

    if len(tokens) > 20: continue
    # print(sample_id, len(lsent))
    count +=1
    result_str += f'{sample_id} '
    sample_range.append(sample_id)
    

sst_dataset = load_sst_treebank(sample_range, verbose=False)
sst_dataset = sst_dataset['train']
tree_dataset = SSTDataset(mode='train')
vocab_words = list(tree_dataset.vocab.keys())


In [7]:
print('A few constrastive conjunction sentences')
print(reduce( lambda x,y: str(x) + ' '+ str(y), list(sst_dataset['sentence'].keys())))
# sample_range = [int(ns) for ns in '138 259 324 385 413 851 1469 1518 1532 1614 1716 2086 2555 3287 3423 3427 3437 3493 3617 4328 4634 4724 5177 5356 5676 5703 5734 6072 6347 6593 7349 7411 7560 8015 8145 8231'.split(' ')]
# sample_range

A few constrastive conjunction sentences
138 259 324 385 413 658 851 1469 1518 1614 1716 2086 2351 2509 2555 3287 3423 3427 3493 3617 4328 4634 4724 5177 5293 5356 5602 5703 5734 6072 6347 6593 7147 7349 7411 7560 8015 8145


## Sample wise consideration of query contributions

In [8]:
####################################################################################
# Custum parameter
attribution_mode =   'corr(q,f)' #  'attribution'#
weight_mode =   'occlusion' #  'significance-1' #  'shapley'   # 
# datamode=  'synthetic'# 'sst_treebank'  # 'sst_huggingface' # 
# neg_tokens_hars_bool= True
neg_tokens = ['[CLS]', '[SEP]', ',', '.', '_', '-', "'" ]
load_harsanyi = False
load_queries = False
neg_tokens_hars_bool = True
parallelize  = False
save_results = False
# comp_subset_attr = False
### "strange it is, but delightfully so." --> sample 385
### "topics that could make a sailor blush - but lots of laughs." --> 1518
sample_id =  385 # 1518
sample_range = [sample_id]
# sample_range = [int(ns) for ns in '138 259 324 385 413 851 1469 1518 1532 1614 1716 2086 2555 3287 3423 3427 3437 3493 3617 4328 4634 4724 5177 5356 5676 5703 5734 6072 6347 6593 7349 7411 7560 8015 8145 8231'.split(' ')]
# hack = False
####################################################################################
# fix parameter
harsanyi_maxorder=4
max_and_order = 3
max_indexdist = 1
max_setsize =  float('inf')
query_mode = 'conj. disj. (neg. disj.) reasonably mixed' # 'conj. disj. neg. reasonably mixed' # 'set conjuction'
result_folder = 'intermediate_results/query_search_algo3/'
###########################################################################################

for sample_id in sample_range:
    ####################################################################
    tree_dataset = SSTDataset(mode='train')
    sst_dataset = load_sst_treebank([sample_id])
    vocab_words = list(tree_dataset.vocab.keys())
    ## Process input
    tree_sample = tree_dataset[sample_id]
    try:
        Xnids, Ynids, but_treeid, mask, lsent, sentence, sent_label, Xlabels, Ylabels = test_contr_conj(tree_sample, vocab_words, verbose=False)
    except:
        raise ValueError(f'sample {sample_id} is not a contrastive conjuction.')
                         
    assert sst_dataset['train']['sentence'][sample_id] == sentence
    
    harsdiv_name = f'harsanyi_div_maxorder-{harsanyi_maxorder}_sampleid-{sample_id}_datamode-sst_treebank.pkl'
    queries_name = f'all_queries-{sample_id}_max_and_order-{max_and_order}_datamode-sst_treebank_attribution_mode-{attribution_mode}_query_mode-{query_mode}_harsanyi_maxorder-{harsanyi_maxorder}.pkl'
    
    sample = tokenizer(sentence, return_tensors="pt")
    tokens = tokenizer.convert_ids_to_tokens(sample['input_ids'].squeeze())
    
    print(f'"{sentence}"', 'prediction is', sst_model(**sample)['logits'].argmax().item())
    
    neg_tokens_ids = [idn for idn, tok in enumerate(tokens) if tok in neg_tokens]
    
    if max_setsize > len(tokens):
        max_setsize = len(tokens)
        
    ##################################################################################
    # setup explainer
    if sst_model(**sample)['logits'].argmax().item() == 0:
        target = torch.tensor([1,-1])
    else:
        target = torch.tensor([-1,1])
        
    explainer = BERTSymbXAI(sample=sample,
                            target=target,
                            model=sst_model,
                            embeddings=pretrained_embeddings)
    ############################################################################################
    ## Set up the explanation framework, 
    # 1) Harsanyi dividends
    if load_harsanyi:
        try:
            hars_div = pickle.load(open(result_folder + harsdiv_name, 'rb'))
        except:
            print(f'could not load the Harsanyi dividends for {sample_id}')
            continue
    else:
        hars_div = comp_all_harsanyi_sst(explainer, harsanyi_maxorder=harsanyi_maxorder, neg_tokens=None if not neg_tokens_hars_bool else neg_tokens_ids)
        if save_results: pickle.dump(hars_div, open(result_folder + harsdiv_name, 'wb'))
    
    if load_queries:
        try:
            all_queries = pickle.load(open(result_folder + queries_name, 'rb'))
        except:
            print(f'could not load the queries for {sample_id}')
            continue
    else:
        # 2) Queries
        all_queries = setup_queries(explainer.node_domain, 
                                    tokens,
                                    max_and_order, 
                                    max_setsize=max_setsize, 
                                    max_indexdist=max_indexdist, 
                                    mode=query_mode,
                                    neg_tokens=neg_tokens_ids )
        # 3) Weight function 
        weight_vec = calc_weights(weight_mode, hars_div, all_queries)
        ##########################################################################################
        
        start = time.time()
        if attribution_mode == 'corr(q,f)':
            calculation_fct = calc_corr
        else:
             calculation_fct = calc_attr_supp
        
        if parallelize:
            pool = multiprocessing.Pool(14)
            args = [(q, hars_div, weight_vec) for q in all_queries]
            all_queries = pool.map(calculation_fct, args)
            pool.close()
        else:
            for q in tqdm(all_queries, desc=f'Query Attribution of {attribution_mode}.'):
               q = calculation_fct((q, hars_div, weight_vec))
                
        print('with' if parallelize else 'without', f'paralyzation it took {time.time() - start} seconds.')
        if save_results: pickle.dump(all_queries, open(result_folder + queries_name, 'wb'))

"strange it is, but delightfully so." prediction is 1


Query Attribution of corr(q,f).: 100%|██████████| 1300/1300 [00:00<00:00, 3461.81it/s]

without paralyzation it took 0.376384973526001 seconds.


# Look at some qualitative results

In [9]:
nb_top = 4
nb_flop=0
# pc_top = 100
vismode = 'show all'
fontcolor = 'white'
file_str =  f'quali_table_search_exhaustive_sample-{sample_id}_max_and_order-{max_and_order}_datamode-sst_treebank_querymode-{query_mode}.png'
all_outs = {weight_mode: {}}

out_color_vals, _, out_keys = plot_quali_table(sentence, 
                                               tokens, 
                                               {weight_mode: all_queries}, 
                                               vismode, 
                                               nb_top=nb_top, 
                                               nb_flop=nb_flop, 
                                               file_str=file_str, 
                                               pc_top=100, 
                                               fontcolor=fontcolor
                                              )

52047 bytes written to file /Users/thomasschnake/Research/Projects/symbolic_xai_cleaned/symbolicXAI_github/notebooks/intermediate_results/query_search_algo/quali_table_search_exhaustive_sample-385_max_and_order-3_datamode-sst_treebank_querymode-conj. disj. (neg. disj.) reasonably mixed.png


occlusion
¬(is strange it) ∧ delightful
¬##ly ∧ delightful ∧ ¬(is strange it)
delightful ∧ ¬(strange it)
¬##ly ∧ delightful ∧ ¬(strange it)


## Loading and evaluating the query contributinos from the files

In [12]:
attribution_mode =  'cov(q,f)'  #  'attribution'#
weight_mode =   'occlusion' #  'significance-1' #  'shapley'   # 
query_mode = 'conj. disj. (neg. disj.) reasonably mixed' 
results_dir = 'intermediate_results/query_search_algo3/'
harsanyi_maxorder = 4
# neg_ids = [1614, 1716, 3437, 5703, 8015]
verbose = True
# sample_range = [ids for ids in sample_range if ids not in neg_ids]
# sample_range = [385]
sample_range = [int(ns) for ns in '138 259 324 385 413 851 1469 1518 1532 1614 1716 2086 2555 3287 3423 3427 3437 3493 3617 4328 4634 4724 5177 5356 5676 5703 5734 6072 6347 6593 7349 7411 7560 8015 8145 8231'.split(' ')]
nb_max = 1
nb_min = 1

bool_outs = {}
print('\t\t','max_prese_Y','\t', 'max_absen_X','\t', 'min_absen_Y', '\t','min_prese_X')
for sample_id in sample_range:
    tree_dataset = SSTDataset(mode='train')
    sst_dataset = load_sst_treebank([sample_id])
    ## Process input
    tree_sample = tree_dataset[sample_id]
    
    try:
        Xnids, Ynids, but_treeid, mask, lsent, sentence, sent_label, Xlabels, Ylabels = test_contr_conj(tree_sample, vocab_words, verbose=False)
    except:
        raise ValueError(f'sample {sample_id} is not a contrastive conjuction.')
                         
    assert sst_dataset['train']['sentence'][sample_id] == sentence
    
    queries_name = f'all_queries-{sample_id}_max_and_order-3_datamode-sst_treebank_attribution_mode-{attribution_mode}_query_mode-{query_mode}_harsanyi_maxorder-{harsanyi_maxorder}.pkl'
    try:
        all_queries = pickle.load(open(results_dir + queries_name, 'rb'))
    except FileNotFoundError:
        if verbose: print('no file for:', sample_id)
        continue
    
    sample = tokenizer(sentence, return_tensors="pt")
    tokens = tokenizer.convert_ids_to_tokens(sample['input_ids'].squeeze())

    # print('we loaded', sample_id)

    
    all_attributions = {q.hash: q.attribution for q in all_queries}
    all_str_rep = {q.hash: q.str_rep for q in all_queries}
    queries_sorted = sorted(all_attributions, key=all_attributions.get)[::-1]
    
    max_hashs = queries_sorted[:nb_max]
    min_hashs = queries_sorted[-nb_min:]
    print(sentence)
    try:
        
        Xtokenids = process_treeid2tokenid(Xnids, mask, tokens, lsent, verbose=False)
        Ytokenids =  process_treeid2tokenid(Ynids, mask, tokens, lsent, verbose=False)
        but_tokenids = process_treeid2tokenid([but_treeid], mask, tokens, lsent, verbose=False)
    except RuntimeError:
        print('parsing not possible for', sample_id)
        continue
            
    max_prese_Y = False
    max_absen_X = False
    min_absen_Y = False
    min_prese_X = False
    # if verbose:
    #     print('Xfeats are', [tokens[idf] for idf in Xtokenids])
    #     print('Yfeats are', [tokens[idf] for idf in Ytokenids])
    #     print('but_feats are', [tokens[idf] for idf in but_tokenids])
        
    for hash in max_hashs:
        print('max hash')
        display(HTML(all_str_rep[hash]))
        for feat_set in hash:
            # if verbose: print('feat in max_hash', [tokens[idf] for idf in feat_set])
            
            # check if the query finds tokens in Y important
            if bool(set(Ytokenids).intersection(feat_set)):
                max_prese_Y = True
            
            # check if the query finds tokens the negation of X important
            if bool(set([tid - len(tokens) for tid in Xtokenids]).intersection(feat_set)):
                max_absen_X = True
    
    for hash in min_hashs:
        print('min hash')
        display(HTML(all_str_rep[hash]))
        for feat_set in hash:
            # if verbose: print('feat in min_hash', [tokens[idf] for idf in feat_set])
            # check if the query finds the abcenΩce of Y important
            if bool(set([tid - len(tokens) for tid in Ytokenids]).intersection(feat_set)):
                min_absen_Y = True
                
            # check if the query finds the precence of X important
            if bool(set(Xtokenids).intersection(feat_set)):
                min_prese_X = True
    bool_outs[sample_id] = (max_prese_Y, max_absen_X, min_absen_Y, min_prese_X)
    
    print('id:',sample_id, '\t',max_prese_Y,'\t\t', max_absen_X,'\t\t', min_absen_Y, '\t\t',min_prese_X)
    print('\n')
    

		 max_prese_Y 	 max_absen_X 	 min_absen_Y 	 min_prese_X


KeyboardInterrupt: 